In [17]:
import pandas
import re
from bs4 import BeautifulSoup as bs
import json
import requests
from requests.exceptions import RequestException
import time
import pandas as pd

df = pd.read_csv("t439.csv",index_col=0)


In [9]:
def get_one_page(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

def parse_one_page(html,ticker):
    time = []
    factor = []
    items = re.findall('<td style="text-align:center">(.*)</td>', html)
    n = int(len(items)/2)
    for i in range(11):
        time.append(items[i*2])
        factor.append(items[i*2+1].lstrip("$"))
    dic = {
        "time":time,
        ticker:factor
    }
    return dic

def restoreNumber(lst):
    """
    change the type of the data
    from str to float
    """
    new_lst = []
    for i in lst:
        if i:
            pattern=re.compile(',')
            numList=pattern.split(i)
            numStr=''.join(numList)
            num = float(numStr)
            new_lst.append(num)
        else:
            new_lst.append(i)
    return new_lst

In [10]:
html = get_one_page("https://www.macrotrends.net/stocks/charts/MMM/3m/total-current-liabilities")
dff = pd.DataFrame(parse_one_page(html,"MMM"))
dff

,time,MMM
0,2020,"7,948"
1,2019,"9,222"
2,2018,"7,244"
3,2017,"7,687"
4,2016,"6,219"
5,2015,"7,118"
6,2014,"5,964"
7,2013,"7,498"
8,2012,"6,200"
9,2011,"5,441"


In [11]:
dictionary_c = {}
dictionary_cl = {}
for i in range(len(df)):
    ticker = df["Symbol"][i]
    name = df["name"][i]
    url_c = "https://www.macrotrends.net/stocks/charts/{}/{}/cash-on-hand".format(ticker,name)
    url_cl = "https://www.macrotrends.net/stocks/charts/{}/{}/total-current-liabilities".format(ticker,name)
    #print(url)
    try:
        html_c = get_one_page(url_c)
        html_cl = get_one_page(url_cl)
        
        d_c = parse_one_page(html_c,ticker)
        d_cl = parse_one_page(html_cl,ticker)
        dictionary_c.update(d_c)
        dictionary_cl.update(d_cl)
    except:
        print(ticker)
c = pd.DataFrame(dictionary_c) 
cl = pd.DataFrame(dictionary_cl)

In [13]:
c.to_csv("cash.csv")

In [15]:
cl.to_csv("liability.csv")